In [ ]:
import os
import requests
from dotenv import load_dotenv
from typing import Dict, Any

load_dotenv("/Users/paddy/Documents/Github/Dump-Truck/last-fm-spotify-agent/.env")

LASTFM_BASE_URL = "https://ws.audioscrobbler.com/2.0"
LASTFM_API_KEY = os.getenv("LASTFM_API_KEY")

def _lastfm_get(params: Dict[str, Any]) -> Dict[str, Any]:
    base_params = {
        "api_key": LASTFM_API_KEY,
        "format": "json",
    }
    response = requests.get(LASTFM_BASE_URL, params={**base_params, **params})
    if response.status_code != 200:
        raise Exception(f"HTTP {response.status_code}: {response.text}")
    data = response.json()
    if "error" in data:
        raise Exception(f"Last.fm error {data.get('error')}: {data.get('message')}")
    return data

In [2]:
def get_lastfm_user_info(username: str) -> Dict[str, Any]:
    """
    Fetch Last.fm user profile information.

    Args:
        username (str): Last.fm username.

    Returns:
        Dict[str, Any]: User profile information.
    """
    params = {
        "method": "user.getInfo",
        "user": username,
    }
    return _lastfm_get(params)

def get_artist_info(artist_name: str) -> Dict[str, Any]:
    """
    Fetch metadata for an artist from Last.fm.

    Args:
        artist_name (str): Artist name (e.g., "Cher").

    Returns:
        Dict[str, Any]: Artist information and stats.
    """
    params = {
        "method": "artist.getInfo",
        "artist": artist_name,
        # Optional: "autocorrect": 1,
        # Optional: "lang": "en",
    }
    return _lastfm_get(params)

def get_track_info(artist_name: str, track_name: str) -> Dict[str, Any]:
    """
    Fetch metadata for a track from Last.fm.

    Args:
        artist_name (str): Artist name.
        track_name (str): Track title.

    Returns:
        Dict[str, Any]: Track information and stats.
    """
    params = {
        "method": "track.getInfo",
        "artist": artist_name,
        "track": track_name,
        # Optional: "autocorrect": 1,
        # Optional: "username": "<lfm-username>",
    }
    return _lastfm_get(params)


In [3]:
get_lastfm_user_info("paddyr1990")

{'user': {'name': 'paddyr1990',
  'age': '0',
  'subscriber': '0',
  'realname': 'Paddy ',
  'bootstrap': '0',
  'playcount': '205332',
  'artist_count': '5545',
  'playlists': '0',
  'track_count': '28999',
  'album_count': '11666',
  'image': [{'size': 'small',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/41d3f4e354231b02b0f1e1ba02ef904c.png'},
   {'size': 'medium',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/41d3f4e354231b02b0f1e1ba02ef904c.png'},
   {'size': 'large',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/41d3f4e354231b02b0f1e1ba02ef904c.png'},
   {'size': 'extralarge',
    '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/41d3f4e354231b02b0f1e1ba02ef904c.png'}],
  'registered': {'unixtime': '1255744804', '#text': 1255744804},
  'country': 'Canada',
  'gender': 'n',
  'url': 'https://www.last.fm/user/paddyr1990',
  'type': 'user'}}

In [4]:
get_artist_info("Opeth")


{'artist': {'name': 'Opeth',
  'mbid': 'c14b4180-dc87-481e-b17a-64e4150f90f6',
  'url': 'https://www.last.fm/music/Opeth',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '0',
  'stats': {'listeners': '1159469', 'playcount': '125790876'},
  'similar': {'artist': [{'name': 'Porcupine Tree',

In [ ]:
import os
import requests
from dotenv import load_dotenv
from typing import Dict, Any

load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

def _get_spotify_token() -> str:
    """Get Spotify access token using Client Credentials flow."""
    auth_url = "https://accounts.spotify.com/api/token"
    auth_data = {
        "grant_type": "client_credentials",
        "client_id": SPOTIFY_CLIENT_ID,
        "client_secret": SPOTIFY_CLIENT_SECRET,
    }
    response = requests.post(auth_url, data=auth_data)
    if response.status_code != 200:
        raise Exception(f"Auth failed: {response.text}")
    return response.json()["access_token"]

def _spotify_get(endpoint: str, params: Dict[str, Any] = None) -> Dict[str, Any]:
    """Generic Spotify GET request helper."""
    token = _get_spotify_token()
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    response = requests.get(f"https://api.spotify.com/v1{endpoint}", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"HTTP {response.status_code}: {response.text}")
    return response.json()

import os
import requests
from dotenv import load_dotenv
from typing import Dict, Any

load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

def _get_spotify_token() -> str:
    """Get Spotify access token using Client Credentials flow."""
    auth_url = "https://accounts.spotify.com/api/token"
    auth_data = {
        "grant_type": "client_credentials",
        "client_id": SPOTIFY_CLIENT_ID,
        "client_secret": SPOTIFY_CLIENT_SECRET,
    }
    response = requests.post(auth_url, data=auth_data)
    if response.status_code != 200:
        raise Exception(f"Auth failed: {response.text}")
    return response.json()["access_token"]

def _spotify_get(endpoint: str, params: Dict[str, Any] = None) -> Dict[str, Any]:
    """Generic Spotify GET request helper."""
    token = _get_spotify_token()
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    response = requests.get(f"https://api.spotify.com/v1{endpoint}", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"HTTP {response.status_code}: {response.text}")
    return response.json()


In [3]:
def search_artists_by_genre(genre: str, limit: int = 20) -> Dict[str, Any]:
    """
    Search for artists matching a genre.
    
    Args:
        genre (str): Genre name (e.g., "metal", "black metal", "prog rock").
        limit (int): Max results (1-50, default 20).
    
    Returns:
        Dict[str, Any]: Search results with artist info.
    """
    params = {
        "q": f'genre:"{genre}"',
        "type": "artist",
        "limit": limit,
    }
    return _spotify_get("/search", params=params)

def get_artist_details(artist_id: str) -> Dict[str, Any]:
    """
    Fetch full artist details by Spotify artist ID.
    
    Args:
        artist_id (str): Spotify artist ID (e.g., "0TnOYISbd1XYRBk9myaseg").
    
    Returns:
        Dict[str, Any]: Artist info including genres, popularity, followers, images.
    """
    return _spotify_get(f"/artists/{artist_id}")

def get_artist_top_tracks(artist_id: str, market: str = "US", limit: int = 10) -> Dict[str, Any]:
    """
    Fetch an artist's top tracks.
    
    Args:
        artist_id (str): Spotify artist ID.
        market (str): ISO 3166-1 alpha-2 country code (default "US").
        limit (int): Max tracks (1-50, default 10).
    
    Returns:
        Dict[str, Any]: List of top tracks with audio features.
    """
    params = {
        "market": market,
        "limit": limit,
    }
    return _spotify_get(f"/artists/{artist_id}/top-tracks", params=params)

def get_artist_albums(artist_id: str, limit: int = 20, include_groups: str = "album") -> Dict[str, Any]:
    """
    Fetch an artist's albums/releases.
    
    Args:
        artist_id (str): Spotify artist ID.
        limit (int): Max results (1-50).
        include_groups (str): "album", "single", "compilation", "appears_on" (comma-separated).
    
    Returns:
        Dict[str, Any]: List of albums with release dates, images, track count.
    """
    params = {
        "limit": limit,
        "include_groups": include_groups,
    }
    return _spotify_get(f"/artists/{artist_id}/albums", params=params)


In [4]:
if __name__ == "__main__":
    # Search for metal artists
    results = search_artists_by_genre("metal", limit=5)
    for artist in results["artists"]["items"]:
        print(f"{artist['name']} (ID: {artist['id']})")
        print(f"  Genres: {artist['genres']}")
        print(f"  Followers: {artist['followers']['total']}")
        print()
    
    # Get details for first result
    if results["artists"]["items"]:
        artist_id = results["artists"]["items"][0]["id"]
        details = get_artist_details(artist_id)
        print(f"Full details for {details['name']}:")
        print(f"  Popularity: {details['popularity']}/100")
        print(f"  Genres: {details['genres']}")
        
        # Get top tracks
        top_tracks = get_artist_top_tracks(artist_id)
        print(f"\nTop tracks:")
        for track in top_tracks["tracks"][:3]:
            print(f"  - {track['name']} ({track['popularity']}/100 popularity)")
        
        # Get albums
        albums = get_artist_albums(artist_id)
        print(f"\nAlbums:")
        for album in albums["items"][:3]:
            print(f"  - {album['name']} ({album['release_date']})")


Linkin Park (ID: 6XyY86QOPPrYVGvF9ch6wz)
  Genres: ['nu metal', 'rap metal', 'rock', 'alternative metal']
  Followers: 32341653

Metallica (ID: 2ye2Wgw4gimLv2eAKyk1NB)
  Genres: ['metal', 'thrash metal', 'rock', 'heavy metal', 'hard rock']
  Followers: 32628500

System Of A Down (ID: 5eAWCfyUhZtHHtBdNk56l1)
  Genres: ['nu metal', 'metal', 'alternative metal', 'rap metal', 'rock']
  Followers: 14270479

Deftones (ID: 6Ghvu1VvMGScGpOUJBAHNH)
  Genres: ['nu metal', 'alternative metal', 'rap metal', 'shoegaze', 'rock']
  Followers: 7936593

Guns N' Roses (ID: 3qm84nBOXUEQ2vnTfUTTFC)
  Genres: ['rock', 'glam metal', 'hard rock', 'classic rock']
  Followers: 35661610

Full details for Linkin Park:
  Popularity: 90/100
  Genres: ['nu metal', 'rap metal', 'rock', 'alternative metal']

Top tracks:
  - In the End (90/100 popularity)
  - Numb (88/100 popularity)
  - Somewhere I Belong (78/100 popularity)

Albums:
  - From Zero (Deluxe Edition) (2025-05-16)
  - From Zero (2024-11-15)
  - From Zero